## Richards Equation Toy model -- Trying the Julia Solvers

In [2]:
using DiffEqOperators,  DifferentialEquations,  Plots
using Parameters
using Revise
#from jupyterthemes import jtplot
#jtplot.style(theme=’monokai’, context=’notebook’, ticks=True, grid=False)

In [3]:
FT = Float64

include("Water/water_functions.jl")
include("Water/hydraulic_conductivity_composable.jl")
include("Water/matric_potential_composable.jl")

matric_potential

In [4]:
hydraulic_model = Havercamp{FT}();
matric_model    = vanGenuchten{FT}();

In [17]:
K_sat  = 0.0443 / (3600*100)
porosity = 0.495 # Read in from data base
S_s = 10e-4  # [ m-1] ## Check this value !!

#IC/BC values  - to be specified/calculated via a BC/IC struct, and treated as an attribute of the model.
ν_0 = 0.24
ν_surface = porosity-1e-3
S_l_0 = effective_saturation(porosity, ν_0)
ψ_0 = pressure_head(matric_model,S_l_0,porosity,S_s,ν_0)
println(ψ_0)
κ_0 = hydraulic_conductivity(hydraulic_model, K_sat, S_l_0, ψ_0,0.0)

-1.938733644779751


1.3548894996468134e-9

In [18]:
L  = 1.0 # depth in meters
nknots = 99 #99
dz = -L/(nknots+1)#1000.0/(nknots+1)
knots = range(dz, step=dz, length=nknots)

z = collect(knots)
ord_deriv = 2
ord_approx = 2

2

In [19]:
const Δ = CenteredDifference(ord_deriv, ord_approx, dz, nknots)
const ∇ = CenteredDifference(1, ord_approx, dz, nknots)
const bc = DirichletBC(ν_surface, ν_0)

RobinBC{Float64,Array{Float64,1}}([-0.0, 0.0], 0.494, [-0.0, 0.0], 0.24)

In [27]:
function soil_water(ν,p,t; porosity=porosity, S_s=S_s, HM=hydraulic_model, MM=matric_model, K_sat=K_sat, z=z )
    # Get effective saturation
    ν_ext = bc*ν
    S_l = map(x -> effective_saturation(porosity,x),ν_ext)
    #@show S_l
    # This function calculates pressure head ψ of a soil
    ψ = similar(S_l)
    K = similar(S_l)
    for i in eachindex(ψ)
        ψ[i] = pressure_head(MM, S_l[i],porosity,S_s,ν_ext[i])
    end
    
    # Get hydraulic head
    #h = hydraulic_head(z,ψ)
    
    # Conductivity
    for i in eachindex(ψ)
        K[i] = hydraulic_conductivity(HM,K_sat,S_l[i],ψ[i],0.0)
    end
    #@show K
    dKdz = ∇*K
   
    K[2:end-1]*Δ*ψ + (∇*ψ) .* dKdz + dKdz

end

soil_water (generic function with 1 method)

In [28]:
ν = ν_0*ones(99);
soil_water(ν,1,0);

In [ ]:
tmax = 11*86400.0
prob = ODEProblem(soil_water, ν, (0.0, tmax))
alg = KenCarp4(autodiff=false)
sol = solve(prob, alg, saveat=10*60.0)
#sol_times = [sol(t) for t in 0.0:dt:tmax]

In [11]:
plot(sol[1],z)
for i=1:20:length(sol.u)
    plot!(sol[i],z)
end
plot!(sol[end],z)

UndefVarError: UndefVarError: sol not defined

In [12]:
length(sol.u)

UndefVarError: UndefVarError: sol not defined